In [27]:
# NOTEBOOK IMPORTS
import os, glob, warnings, pickle, re
import numpy as np
from shutil import copyfile, rmtree
from datetime import datetime
from fuzzywuzzy import process

# IMAGE IMPORTS
# from PIL import Image

# GIS IMPORTS
from affinetransformation import *
from affine import Affine
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point, MultiPoint, box
import rasterio as rio
import contextily as cx


# PLOTTING IMPORTS
import matplotlib.pyplot as plt

# CUSTOM UTILITIES
from IndexUtils import * 
from TileUtils import *

Image.MAX_IMAGE_PIXELS = 933120000
warnings.filterwarnings("ignore")
initialize = False

In [2]:
input_dir = r"D:\RECTDNN\uncompress\\"
proc_dir  = r"D:\RECTDNN\processing\2024-05-31_14-05-48\Tiles\\"

In [3]:
stateplanes = gpd.read_file(f"{data_dir}/AAA_ReferenceDatasets/stateplane.shp")

init_databases(f"{data_dir}/AAA_ReferenceDatasets/")

True

In [4]:
# LIST ALL IMAGES IN DIRECTORY
image_files = glob.glob(f"{input_dir}/*")
image_files = [f for f in image_files if 'w' not in os.path.splitext(f)[1]]

# FILTER IMAGES USING HEURISTICS TO GET TILEINDICES
patterns = ["IND", "_1."]
index_files = [file for pattern in patterns for file in glob.glob(input_dir + "\\*" + pattern + "*.*[!w]*")]
filtered_files = [file for file in image_files if len(os.path.basename(file)) < 12]
index_files.extend(filtered_files)

# GET ACTUAL TILES
tiles       = list(set(image_files) - set(index_files))
tiles_bns   = [os.path.basename(tile).split(".")[0] for tile in tiles]

with open(f"{data_dir}/AAA_ReferenceDatasets/IndexCoords.pkl", 'rb') as handle:
    dict = pickle.load(handle)

We create the tile database with the detected tiles

In [5]:
verbose = False
dupped_dict = {}
non_dupped_dict = {}

db = {}

possible_tiles = 0

for fn in dict.keys():

    # GET AFFIEN TRANSFORM FROM CURRENT INDEX
    affine   = Affine(*dict[fn]["output_transform"].flatten()[:6])

    db[findKey(fn)] = []

    possible_tiles = possible_tiles + len(dict[fn]['tile'].keys())
    if verbose:
        print('-' * 10 + fn + '-' * 10 + str(len(dict[fn]['tile'].keys())))
    for i in dict[fn]['tile'].keys():

        # SPLIT NEW LINES AND FILTER THOSE WITH LESS THAN 3 NUMBERS
        text_ori = dict[fn]['tile'][i]['text'].split("\n")
        text = [a for a in text_ori if len("".join(re.findall("\d+", a))) > 3]

        # FIND KEY BASED ON INDEX NAME
        curr_key = findKey(fn)

        # EXTRACT ONE OF THE LIST WITH KEY
        data = process.extractOne(curr_key, text)
        if data is None:
            if verbose:
                print("NOT FOUND "+ " /n ".join(text_ori))
            continue
        found_text, score = data

        # IF SCORE IS GOOD ENOUGH
        if score > 60:

            # FIND INDEX OF MATCHED TEXT IN LINES 
            idx = text.index(found_text)
            
            # FIND ALL NUMBERS IN MATCHED TEXT
            foundnumbers = "".join(re.findall("\d+", text[idx].replace(" ", "")))
            
            # REMOVE ANY CHARACTERS PRIOR TO PARTIAL MATCH TO KEY
            for a in range(len(curr_key), 1, -1):
                currentnumbers = foundnumbers.split(curr_key[-a:])[-1]
                if currentnumbers != foundnumbers:
                    break
            
            # IF WE HAVE EXACTLY 5 CHARACTERS, AND THE LAST IS 8, IT'S LIKELY IT WAS A MISREAD "B"
            if len(currentnumbers) == 5 and (currentnumbers[-1] == "8" or currentnumbers[-1] == "7"):
                currentnumbers = currentnumbers[:-1]

            if verbose:
                print(curr_key + currentnumbers + " | " +foundnumbers + " | " +  " /n ".join(text))

            # CALCULATE COORDS FROM AFFINE 
            bbox = dict[fn]['tile'][i]['bbox']
            left, bottom = affine * (bbox[0], bbox[1])
            right, top   = affine * (bbox[2], bbox[3])

            # PREPARE DICT 
            out_dict = dict[fn]['tile'][i]
            out_dict['coords'] = np.array([left, bottom, right, top])

            # SAVE OUTPUT
            dupped_dict[curr_key + currentnumbers] = out_dict
            
            data = non_dupped_dict.get(curr_key + currentnumbers, None)
            if data is None: 
                non_dupped_dict[curr_key + currentnumbers] = out_dict

            else:
                i=0
                while data is not None:
                    data = non_dupped_dict.get(curr_key + currentnumbers + f"_{i}", None)
                    i = i + 1
                non_dupped_dict[curr_key + currentnumbers + f"_{i}"] = out_dict

            db[findKey(fn)].append(curr_key + currentnumbers)

print(len(non_dupped_dict), len(dupped_dict))  

detected_tiles = pd.DataFrame.from_dict(non_dupped_dict).T
detected_tiles['geometry'] = detected_tiles['coords'].apply(bbox_to_polygon)
detected_tiles_gdf = gpd.GeoDataFrame(detected_tiles)
detected_tiles_gdf = detected_tiles_gdf[['geometry']]

2190 1493


Here we supplement that database with all existing world files...

In [6]:
# READ ALL THE WORLD FILES FROM A DIRECTORY AND CREATE A DATAFRAME
data = read_world_files_from_directory(input_dir)
columns = ['filename', 'line1', 'line2', 'line3', 'line4', 'line5', 'line6']
df = pd.DataFrame(data, columns=columns)

# CONVERT ALL READ PARAMETERS TO NUMERIC
for col in [col for col in df.columns if "line" in col]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# FIND GEOMETRY FOR EACH ROW
df['key']   = df['filename'].apply(findTileKey, db=db)  # FIND THE KEY FOR THE FILENAME
df['key_n'] = pd.to_numeric(df['key'], errors='coerce') # CONVERT IT TO NUMERIC
df["GEOID"] = df["key_n"].apply(getGEOID)               # GET GEOID FOR EACH INDEX
df['county_polygon'] = df["GEOID"].apply(getGeometry)   # USE FIND GEOMETRY FUNCTION

# HEURISTICS - DEFINE WHETHER THE FOUND TRANSFORM IS STATEPLANE 
# BY IT'S SCALE. IF IT'S BIGGER THAN 1, PROBABLY (MOST FILE'S RESOLUTION IS < 0.5 m)
df['STATEPLANE'] = df['line1'] > 1

# INTERSECT DF WITH STATE PLANE SHAPEFILE 
geo_df   = gpd.GeoDataFrame(df, geometry=df['county_polygon']).set_crs("EPSG:3857").to_crs("EPSG:4326")
df_plane = geo_df.overlay(stateplanes, how='intersection')

# CALCULATE EPSG CODE
df['epsg'] = geo_df.apply(getEPSG, df_plane=df_plane, axis=1)
del geo_df, df_plane

df['geotransform'] = df['filename'].apply(getGeotransform, input_dir=input_dir)
df['affine'] = df['geotransform'].apply(getAffine)

In [7]:
webmercator = []

RLNN = None
def notify(mess, level=2):
    if level < 4:
        print(mess)

for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    fn = row['filename'].split(".")[0]
    image_files = glob.glob(f'{os.path.join(input_dir, fn)}*[!w]')

    if pd.isna(row['epsg']):
        notify(f"NO EPSG {fn}")
        webmercator.append([])
        continue

    if len(image_files) == 0:
        notify(f"NO CORRESPONDING IMAGE {fn}")
        webmercator.append([])
        continue
    bounds, RLNN = findBounds(image_files[0], model=RLNN, verbose=False, device="cuda")

    if len(bounds[0]) == 0:
        notify(f"NO BOUNDS FOUND {image_files[0]}")
        webmercator.append([])
        continue

    bbox = bounds[0].boxes.xyxy.numpy()[0]

    in_crs  = rio.crs.CRS.from_string(f"{row['epsg']}")
    out_crs = rio.crs.CRS.from_epsg(f"3857")

    left, bottom = row['affine'] * (bbox[0], bbox[1])
    right, top   = row['affine'] * (bbox[2], bbox[3])

    new_bbox = rio.warp.transform_bounds(in_crs, out_crs, left, bottom, right, top)
    webmercator.append(new_bbox)
    # print(bbox, [bottom, left, top, right], new_bbox)

df["webmerc"] = webmercator

  0%|          | 0/178 [00:00<?, ?it/s]

NO BOUNDS FOUND D:\RECTDNN\uncompress\48201C0195M.tif
NO BOUNDS FOUND D:\RECTDNN\uncompress\48201C0415M.tif
NO BOUNDS FOUND D:\RECTDNN\uncompress\48201C0970M.tif
NO BOUNDS FOUND D:\RECTDNN\uncompress\48201C1030L.tif


In [8]:
# Apply the function to the 'bbox' column
df['geometry'] = df['webmerc'].apply(bbox_to_polygon)
gdf = gpd.GeoDataFrame(df)
gdf.dropna(axis=0)

for key in gdf.keys():
    if key == 'geometry':
        continue
    gdf[key] = gdf[key].astype(str)

In [9]:
gdf.to_file(r"D:\RECTDNN\WorldFiles.shp")
detected_tiles_gdf.to_file(r"D:\RECTDNN\Detected.shp")

In [96]:
FANN_prior = None
RLNN_prior = None
model = None

color=False

plot_params = {"labels" : False, }
YOLO_params = {
    "device"  : "cuda",     "find_text" : False, 
    "keyed_text"  : False,  "verbose" : False,  "get_data"  : False,
    "target_size" : 1920,   "plot_params" : plot_params
}

# IF WE'VE ALREADY STARTED PROCESSING THINGS, LOAD STATE DICT
if os.path.exists(f'{proc_dir}results.pkl'):
    with open(f'{proc_dir}results.pkl', 'rb') as handle:
        results_struct = pickle.load(handle)
else:
    results_struct = {}

# LOOP FOR EACH TILE
for i, tile in tqdm(enumerate(tiles), total=len(tiles)):
    
    # DEFINE WHERE GEOREFERENCED TILE IS SAVED
    out_fn = os.path.join(proc_dir, os.path.basename(tile).split(".")[0] + "_FANN.png")

    # HAVE WE ALREADY PROCESSED THIS TILE? IF SO, SKIP
    if results_struct.get(tile, None):
        print(f"Found in dict, skipping {tile}")
        continue

    # WHICH VERSION OF FANN ARE WE USING
    if color:
        # RUN AND SAVE PREPROCESSING FANN AND RLNN
        prepped_fn = os.path.join(proc_dir, os.path.basename(tile).split(".")[0] + "_prepped.png")                      # FILE NAME
        prepped, FANN_prior, RLNN_prior, bounds = findStreetCorners_colorPrep(tile, FANN=FANN_prior, RLNN=RLNN_prior)   # RUN FANN_PRIOR AND RLNN
        Image.fromarray(prepped).save(prepped_fn) # SAVE

        # RUN FANN
        results, model = runYOLO_Text(prepped_fn, model=model, model_weights=f"{data_dir}FANN/YOLO/051624.pt", save_dir=out_fn, **YOLO_params)
        results_struct[tile] = {"results" : results, "bounds" : bounds}

    else:
        # RUN FANN
        results, model = runYOLO_Text(tile, model=model, model_weights=f"{data_dir}FANN/YOLO/051624_bw.pt", save_dir=out_fn, **YOLO_params)
        results_struct[tile] = {"results" : results, "bounds" : bounds}

    # SAVE EVERY N ITERATIONS AND LAST
    if i % 10 == 0 or i == len(tiles) - 1:
        with open(f'{proc_dir}results.pkl', 'wb') as handle:
            pickle.dump(results_struct, handle, protocol=pickle.HIGHEST_PROTOCOL)

  0%|          | 0/2251 [00:00<?, ?it/s]

Found in dict, skipping D:\RECTDNN\uncompress\4800350550B.jpg


KeyboardInterrupt: 

In [80]:
def bboxTransformToCRS(transform, image):
    rev_y_axis = np.array([[1, 0, 0],
                        [0,-1, 0],
                        [0, 0, 1]])

    # move = original_homography @ np.array([0, image_t.shape[0], 0])
    translation = np.eye(3)
    translation[1, 2] = image.shape[0]

    return transform @ translation @ rev_y_axis


def bbox_to_coords_realworld(bbox):
    '''  
    x1, y1, x2, y2 = bbox

    x_min = np.min([x1, x2])    
    x_max = np.max([x1, x2])    
    y_min = np.min([y1, y2])    
    y_max = np.max([y1, y2])  
    '''

    x_min, y_min, x_max, y_max = bbox

    xs = [x_min, x_min, x_max, x_max]
    ys = [y_max, y_min, y_min, y_max]
    return xs, ys

def bbox_to_coords_raster(bbox):
    '''
    x1, y1, x2, y2 = bbox

    x_min = np.min([x1, x2])    
    x_max = np.max([x1, x2])    
    y_min = np.min([y1, y2])    
    y_max = np.max([y1, y2])    

    
    '''
    x_min, y_min, x_max, y_max = bbox

    xs = [x_min, x_min, x_max, x_max]
    ys = [y_min, y_max, y_max, y_min]
    return xs, ys

bad = 0
RLNN = None

detected_tiles['reference_name'] = detected_tiles.index
list_tiles = list(detected_tiles_gdf.index)

tile_coords = {}
debug_struct = {}
for i, tile in tqdm(enumerate(tiles), total=len(tiles)):
    
    

    # KEEP NUMERIC PART OF TILE NAME
    p = re.sub(r"[^0-9]", "", os.path.basename(tile))
    
    # PERFORM MATCHING
    match, score = process.extractOne(tiles_bns[i], list_tiles)

    out_fn = os.path.join(proc_dir, tiles_bns[i] + ".tif")
    
    if score > 90:
        image = np.asarray(Image.open(tile))

        if np.max(image) < 255:
            image = image * 255
            image = image.astype(np.uint8)

        tile_coords[p] = detected_tiles.loc[match]
        bounds, RLNN = findBounds(tile, RLNN)
        
        if len(bounds[0]) < 1:
            print(f"COULD NOT FIND BOUNDS, USING IMAGE EXTENTS {tile}")
            
            bbox = [0, 0, image.shape[0], image.shape[1]]
        else:
            bbox = bounds[0].boxes.xyxy.numpy().astype(np.int32)[0].flatten()
        
        real_x, real_y = bbox_to_coords_realworld(tile_coords[p]["coords"])
        pic_x, pic_y   = bbox_to_coords_raster(bbox)

        # transform = similarityTransformation(pic_x, pic_y, real_x, real_y)
        affine = affineTransformation(pic_x, pic_y, real_x, real_y)
        transform = bboxTransformToCRS(affine.matrix, image)

        # print(*transform.flatten()[:6])

        debug_struct[p] = {
            "real_x" : real_x, "real_y" : real_y,
            "pic_x"  : pic_x,  "pic_y"  : pic_y,
            "affine" : affine, "transform" : transform

        }

        with rio.open(out_fn, 'w',
            driver='GTiff', count=1, dtype=image.dtype,
            height=image.shape[0], width=image.shape[1],
            crs=f'EPSG:3857',
            transform=rio.Affine(*transform.flatten()[:6])) as dst:
                dst.write(image, 1) 
                
    else:
        tile_coords[p] = None
        bad = bad + 1

  0%|          | 0/2251 [00:00<?, ?it/s]

COULD NOT FIND BOUNDS, USING IMAGE EXTENTS D:\RECTDNN\uncompress\485470A_1.tif
COULD NOT FIND BOUNDS, USING IMAGE EXTENTS D:\RECTDNN\uncompress\4802960125C.jpg
COULD NOT FIND BOUNDS, USING IMAGE EXTENTS D:\RECTDNN\uncompress\4802870260E.jpg
COULD NOT FIND BOUNDS, USING IMAGE EXTENTS D:\RECTDNN\uncompress\4800450042B.jpg
COULD NOT FIND BOUNDS, USING IMAGE EXTENTS D:\RECTDNN\uncompress\4803070015D.jpg
COULD NOT FIND BOUNDS, USING IMAGE EXTENTS D:\RECTDNN\uncompress\4854560010D.jpg
COULD NOT FIND BOUNDS, USING IMAGE EXTENTS D:\RECTDNN\uncompress\4802660225.tif
COULD NOT FIND BOUNDS, USING IMAGE EXTENTS D:\RECTDNN\uncompress\4854560015.tif
COULD NOT FIND BOUNDS, USING IMAGE EXTENTS D:\RECTDNN\uncompress\4802670002C.tif
COULD NOT FIND BOUNDS, USING IMAGE EXTENTS D:\RECTDNN\uncompress\4800450032C.jpg
COULD NOT FIND BOUNDS, USING IMAGE EXTENTS D:\RECTDNN\uncompress\4802660225C.tif
COULD NOT FIND BOUNDS, USING IMAGE EXTENTS D:\RECTDNN\uncompress\4854560015B.jpg
COULD NOT FIND BOUNDS, USING IMA

In [81]:
results

{0: {'bbox': array([     391.17,        1964,      433.63,      2011.1]),
  'data': <PIL.Image.Image image mode=1 size=42x48>,
  'text': None,
  'keyed_text': None,
  'confidence': 0.46481782},
 1: {'bbox': array([     706.26,      879.28,      747.67,      923.73]),
  'data': <PIL.Image.Image image mode=1 size=41x44>,
  'text': None,
  'keyed_text': None,
  'confidence': 0.43216625},
 2: {'bbox': array([     461.75,      1361.4,      494.56,      1393.7]),
  'data': <PIL.Image.Image image mode=1 size=33x32>,
  'text': None,
  'keyed_text': None,
  'confidence': 0.40378115},
 3: {'bbox': array([     262.43,      1637.4,      290.59,      1668.4]),
  'data': <PIL.Image.Image image mode=1 size=28x31>,
  'text': None,
  'keyed_text': None,
  'confidence': 0.40176573},
 4: {'bbox': array([     427.47,      1321.8,      460.93,      1358.9]),
  'data': <PIL.Image.Image image mode=1 size=33x37>,
  'text': None,
  'keyed_text': None,
  'confidence': 0.37235332},
 5: {'bbox': array([     296.9

In [37]:
transform.matrix

array([[   -0.13152,          -0, -1.0944e+07],
       [          0,    -0.13152,  3.4161e+06],
       [          0,           0,           1]])